#  **3268. Find Overlapping Shifts II**

**Table: EmployeeShifts**
``` sql
+------------------+----------+
| Column Name      | Type     |
+------------------+----------+
| employee_id      | int      |
| start_time       | datetime |
| end_time         | datetime |
+------------------+----------+
```
(employee_id, start_time) is the unique key for this table.

This table contains information about the shifts worked by employees, including the start time, and end time.

Write a solution to analyze overlapping shifts for each employee. Two shifts are considered overlapping if they occur on the same date and one shift's end_time is later than another shift's start_time.

For each employee, calculate the following:

The maximum number of shifts that overlap at any given time.

The total duration of all overlaps in minutes.

Return the result table ordered by employee_id in ascending order.

The query result format is in the following example.

 
**Example:**

**Input:**

**EmployeeShifts table:**
``` sql
+-------------+---------------------+---------------------+
| employee_id | start_time          | end_time            |
+-------------+---------------------+---------------------+
| 1           | 2023-10-01 09:00:00 | 2023-10-01 17:00:00 |
| 1           | 2023-10-01 15:00:00 | 2023-10-01 23:00:00 |
| 1           | 2023-10-01 16:00:00 | 2023-10-02 00:00:00 |
| 2           | 2023-10-01 09:00:00 | 2023-10-01 17:00:00 |
| 2           | 2023-10-01 11:00:00 | 2023-10-01 19:00:00 |
| 3           | 2023-10-01 09:00:00 | 2023-10-01 17:00:00 |
+-------------+---------------------+---------------------+
```
**Output:**
``` sql
+-------------+---------------------------+------------------------+
| employee_id | max_overlapping_shifts    | total_overlap_duration |
+-------------+---------------------------+------------------------+
| 1           | 3                         | 600                    |
| 2           | 2                         | 360                    |
| 3           | 1                         | 0                      |
+-------------+---------------------------+------------------------+
```
**Explanation:**

**Employee 1 has 3 shifts:**
- 2023-10-01 09:00:00 to 2023-10-01 17:00:00
- 2023-10-01 15:00:00 to 2023-10-01 23:00:00
- 2023-10-01 16:00:00 to 2023-10-02 00:00:00

The maximum number of overlapping shifts is 3 (from 16:00 to 17:00). 

The total overlap duration is: 
- 2 hours (15:00-17:00) between 1st and 2nd shifts 
- 1 hour (16:00-17:00) between 1st and 3rd shifts 
- 7 hours (16:00-23:00) between 2nd and 3rd shifts 
- Total: 10 hours = 600 minutes

**Employee 2 has 2 shifts:**
- 2023-10-01 09:00:00 to 2023-10-01 17:00:00
- 2023-10-01 11:00:00 to 2023-10-01 19:00:00
- The maximum number of overlapping shifts is 2. 
- The total overlap duration is 6 hours (11:00-17:00) = 360 minutes.

Employee 3 has only 1 shift, so there are no overlaps.

The output table contains the employee_id, the maximum number of simultaneous overlaps, and the total overlap duration in minutes for each employee, ordered by employee_id in ascending order.

**Solution-1:**
``` sql
with cte as(
select e1.*, count(e2.employee_id) over(partition by e1.employee_id, e1.start_time) as overlap_shifts,
case when e1.start_time =  e2.start_time  then 0 
else timestampdiff(minute,e2.start_time, e1.end_time) end as overlap
from employeeshifts e1
left join employeeshifts e2
on e1.employee_id = e2.employee_id and date(e1.start_time) and date(e2.start_time)
and date(e1.end_time) and date(e2.end_time)
and e1.end_time >= e2.start_time and e1.start_time <= e2.start_time)

select employee_id, max(overlap_shifts) as max_overlapping_shifts, sum(overlap) as total_overlap_duration 
from cte
group by employee_id
order by 1 asc

**Solution-2:**
``` sql
# Write your MySQL query statement below
WITH shift_changes AS (
    SELECT employee_id, DATE(start_time) AS shift_date, start_time AS change_time, 1 AS shift_change
    FROM EmployeeShifts
    UNION ALL
    SELECT employee_id, DATE(end_time) AS shift_date, end_time AS change_time, -1 AS shift_change
    FROM EmployeeShifts
),
concurrent_shifts AS (
    SELECT 
        employee_id, 
        shift_date, 
        change_time, 
        SUM(shift_change) OVER (PARTITION BY employee_id, shift_date ORDER BY change_time) AS concurrent_count
    FROM shift_changes
),
max_overlaps AS (
    SELECT 
        employee_id, 
        MAX(concurrent_count) AS max_overlapping_shifts
    FROM concurrent_shifts
    GROUP BY employee_id
),
overlap_durations AS (
    SELECT 
        e1.employee_id,
        SUM(
            GREATEST(
                TIMESTAMPDIFF(MINUTE, 
                    GREATEST(e1.start_time, e2.start_time),
                    LEAST(e1.end_time, e2.end_time)
                ),
                0
            )
        ) AS total_overlap_duration
    FROM EmployeeShifts e1
    JOIN EmployeeShifts e2 ON e1.employee_id = e2.employee_id AND e1.start_time < e2.start_time
    WHERE e1.end_time > e2.start_time
    GROUP BY e1.employee_id
)
SELECT 
    DISTINCT e.employee_id,
    COALESCE(m.max_overlapping_shifts, 1) AS max_overlapping_shifts,
    COALESCE(o.total_overlap_duration, 0) AS total_overlap_duration
FROM 
    EmployeeShifts e
LEFT JOIN 
    max_overlaps m ON e.employee_id = m.employee_id
LEFT JOIN 
    overlap_durations o ON e.employee_id = o.employee_id
ORDER BY 
    e.employee_id;